<a href="https://colab.research.google.com/github/ferugit/speaker-recognition-exploration/blob/testing/Extract_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
start = 1
end = 1247
batch_size = 20
for s in range(start, end + 1, batch_size):
  batch_end = min(s + batch_size - 1, end)
  print(f"batch start at {s}, and finish at {batch_end}")

In [ ]:
!unzip /content/clips.zip -d /content/clips

In [ ]:
!pip install transformers

In [ ]:
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
import torch
import pandas as pd
import os
import numpy as np

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')

# read database excel file
df = pd.read_csv('path_where_data_information_are_stored', header = 0, sep = '\t')
A = df['Speaker_ID'].value_counts()

unique_speaker_ids = A.index
string_to_number = {string: idx for idx, string in enumerate(unique_speaker_ids)}

# Map the 'Speaker_ID' column using the dictionary
prefix = 'path_to_the_data_file'
df['Speaker_ID'] = df['Speaker_ID'].map(string_to_number)

B = df['Speaker_ID'].value_counts()

# Assuming you have a DataFrame called 'df' with a column 'path' containing the file paths
audio_files = df['Filename'].tolist()

# Add the prefix to each file path and keep only .wav files
audio_files = [os.path.join(prefix, f) for f in audio_files if f.endswith(".wav")]

n=1
start = 1
end = 1247
batch_size = 20
for s in range(start, end + 1, batch_size):
    batch_end = min(s + batch_size - 1, end)
    embeddings = []
    audios = []
    for audio in audio_files[s:batch_end+1]:
# Load and preprocess the audio files
        audio_file1 = audio

        waveform1, _ = torchaudio.load(audio_file1)


        waveform1 = waveform1.squeeze()  # Remove the extra dimension

        inputs1 = feature_extractor(waveform1, return_tensors="pt")

    # Extract embeddings for the audio files
        embeddings1 = model(**inputs1).embeddings
        embeddings.append(embeddings1)
        audios.append(audio)
    data = {'name':audios,'embeddings':[embedding.detach().numpy() for embedding in embeddings]}
    df = pd.DataFrame(data)
    path1 = "/content/model1embeddings" + str(s) + ".tsv"
    df.to_csv(path1, sep='\t', index=False)
    print("Data has been successfully written to.")

In [ ]:
import pandas as pd
import glob

# Delete the Data Information File Before
# Get the list of all .tsv files in the directory
file_list = glob.glob("*.tsv")

# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate over each file in the list
for file in file_list:
    # Read the current file into a DataFrame
    df = pd.read_csv(file, sep='\t')

    # Append the data to the merged_data DataFrame
    merged_data = merged_data.append(df, ignore_index=True)

# Write the merged data to a new .tsv file
merged_data.to_csv("merged_file.tsv", sep='\t', index=False)


In [ ]:
!unzip /content/clips3.zip -d /content/clips3
!pip install speechbrain

In [ ]:
from speechbrain.pretrained import EncoderClassifier

classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

# read database excel file
df = pd.read_csv('path_where_data_information_are_stored', header = 0, sep = '\t')
A = df['Speaker_ID'].value_counts()

unique_speaker_ids = A.index
string_to_number = {string: idx for idx, string in enumerate(unique_speaker_ids)}


# Map the 'Speaker_ID' column using the dictionary
prefix = 'path_to_the_data_file'
df['Speaker_ID'] = df['Speaker_ID'].map(string_to_number)

B = df['Speaker_ID'].value_counts()

# Assuming you have a DataFrame called 'df' with a column 'path' containing the file paths
audio_files = df['Filename'].tolist()

# Add the prefix to each file path and keep only .wav files
audio_files = [os.path.join(prefix, f) for f in audio_files if f.endswith(".wav")]

n=1
start = 1
end = 1247
batch_size = 20

for s in range(start, end + 1, batch_size):
    batch_end = min(s + batch_size - 1, end)
    embeddings = []
    audios = []
    for audio in audio_files[s:batch_end+1]:

# Load and preprocess the audio files
        signal  =classifier.load_audio(audio)
        embeddings1 = classifier.encode_batch(signal, normalize=True)


        embeddings.append(embeddings1)
        audios.append(audio)
    data = {'name':audios,'embeddings':embeddings}
    df = pd.DataFrame(data)
    path1 = "/content/model2embeddings" + str(s) + ".tsv"
    df.to_csv(path1, sep='\t', index=False)
    print("Data has been successfully written to.")

In [ ]:
# Set the directory path where the WAV files are located
directory = "/content"

# Get a list of all files in the directory
file_list = os.listdir(directory)

# Iterate through the file list and delete the WAV files
for file_name in file_list:
    if file_name.endswith(".wav"):
        file_path = os.path.join(directory, file_name)
        os.remove(file_path)

In [ ]:

# Delete the Data Information File Before
# Get the list of all .tsv files in the directory
file_list = glob.glob("*.tsv")

# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate over each file in the list
for file in file_list:
    # Read the current file into a DataFrame
    df = pd.read_csv(file, sep='\t')

    # Append the data to the merged_data DataFrame
    merged_data = merged_data.append(df, ignore_index=True)

# Write the merged data to a new .tsv file
merged_data.to_csv("merged__file.tsv", sep='\t', index=False)
